In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("/Users/vishaalyalamanchali/Desktop/CS178/")
import mltools as ml
import time

In [2]:
X = np.genfromtxt('/Users/vishaalyalamanchali/Desktop/CS178/uci-cs178-spr20/X_train.txt', delimiter=None) 
Y = np.genfromtxt('/Users/vishaalyalamanchali/Desktop/CS178/uci-cs178-spr20/Y_train.txt', delimiter=None) 
X,Y = ml.shuffleData(X,Y)

In [3]:
# [X_train, X_val, Y_train, Y_val] = ml.splitData(X,Y,0.50)
from sklearn import *
X_train, X_val, Y_train, Y_val = model_selection.train_test_split(
    X, Y, test_size=0.2, random_state=41)
print(X_train.shape, X_val.shape)

(160000, 14) (40000, 14)


In [4]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

rfm = RandomForestClassifier(n_estimators=1000, min_samples_leaf=4, n_jobs=-1, oob_score=True)
rfm.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [5]:
y_valpredictions = rfm.predict(X_val)
print("VALIDATION DATASET CLASSIFICATION REPORT: RFM VALIDATION")
print(classification_report(Y_val,y_valpredictions))
random_roc = roc_auc_score(Y_val, rfm.predict_proba(X_val)[:,1])
print("validation roc:", random_roc)

VALIDATION DATASET CLASSIFICATION REPORT: RFM VALIDATION
              precision    recall  f1-score   support

         0.0       0.76      0.88      0.82     26263
         1.0       0.68      0.48      0.56     13737

    accuracy                           0.74     40000
   macro avg       0.72      0.68      0.69     40000
weighted avg       0.74      0.74      0.73     40000

validation roc: 0.7867618597817319


In [6]:
extra_tree_classifier = ExtraTreesClassifier(n_estimators=500, max_depth=200, min_samples_split=10, min_samples_leaf=4, max_features='log2', n_jobs=-1)
extra_tree_classifier_fulldata = ExtraTreesClassifier(n_estimators=500, max_depth=50, min_samples_split=10, min_samples_leaf=4, max_features='log2', n_jobs=-1)

print("training started")
starting_time = time.time()
extra_tree_classifier.fit(X_train, Y_train)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))


extra_tree_classifier_fulldata.fit(X, Y)
extra_tree_classifier_roc = roc_auc_score(Y_val, extra_tree_classifier.predict_proba(X_val)[:,1])
print("validation roc:", extra_tree_classifier_roc)

new_extra_tree_classifier_roc = roc_auc_score(Y_val, extra_tree_classifier_fulldata.predict_proba(X_val)[:,1])
print("validation roc:", new_extra_tree_classifier_roc)

training started
training finished, took 37.13639712333679 seconds
validation roc: 0.7808260285762562
validation roc: 0.918354602458396


In [7]:
Xte = np.genfromtxt('/Users/vishaalyalamanchali/Desktop/CS178/uci-cs178-spr20/X_test.txt', delimiter=None)
Yte = np.vstack((np.arange(Xte.shape[0]), rfm.predict_proba(Xte)[:,1])).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

In [8]:
import time
from sklearn.svm import SVC 
print("training started")
starting_time = time.time()
clf = SVC(kernel= 'precomputed',C = 1e5, probability = True)
clf.fit(X_train, Y_train)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

clf_roc = roc_auc_score(Y_val, clf.predict_proba(X_val)[:,1])
print("validation roc:", clf_roc)

training started


ValueError: Precomputed matrix must be a square matrix. Input is a 160000x14 matrix.